In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime
from dateutil.relativedelta import relativedelta
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.express as px


In [23]:
df_hist_energia = pd.read_csv('..//Dados de Consumo Aneel//dados_consumo_agrupados_nivel_estado.csv', sep = ';', decimal = ',')
df_hist_energia['MesReferencia'] = pd.to_datetime(df_hist_energia['MesReferencia'])
df_hist_energia['TicketMédio'] = df_hist_energia['Receita']/df_hist_energia['QtdConsumidores']
df_hist_energia['TakeRate'] =  df_hist_energia['EnergiaConsumida']/ df_hist_energia['Receita']
df_hist_energia['ConsumoMédio'] =  df_hist_energia['EnergiaConsumida']/ df_hist_energia['QtdConsumidores']

In [24]:
chaves = ['Estado','MesReferencia']
df_hist_energia_target = df_hist_energia[chaves + ['EnergiaInjetada']].copy()
df_hist_energia_target.rename(columns = {'EnergiaInjetada':'Target'}, inplace=True)

df_hist_energia_target['MesReferencia']=  df_hist_energia_target['MesReferencia'].map(lambda x: x - relativedelta(months= 3) )

df_hist_energia = df_hist_energia.merge(df_hist_energia_target, on = chaves, how = 'left')
df_hist_energia.sort_values(by = ['Estado','MesReferencia'], inplace = True, ignore_index= True)


In [12]:
df_hist_energia.dropna().to_parquet('feature_consumo.parquet')

In [25]:
df_hist_energia = df_hist_energia.rename(columns = {'MesReferencia':"event_timestamp"})

In [17]:
df_hist_energia['event_timestamp'] = df_hist_energia['event_timestamp'].map(lambda x: x.timestamp())

In [26]:
np.datetime64(df_hist_energia['event_timestamp'])

ValueError: Could not convert object to NumPy datetime

In [84]:
df_hist_energia.dropna().to_parquet('feature_consumo.parquet')

In [6]:
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource
from datetime import timedelta

# Define uma entidade para associar as features
estado = Entity(name="Estado", value_type=ValueType.STRING, description="Estado")




lst_features = []

for coluna in  ['EnergiaConsumida', 'EnergiaInjetada',
       'QtdConsumidores', 'Receita', 'TicketMédio', 'TakeRate', 'ConsumoMédio',
       'Target']:
    lst_features.append(Feature(name=coluna, dtype=ValueType.FLOAT))

# Define a source dos dados
parquet_source = FileSource(
    event_timestamp_column='event_timestamp',
    path="feature_consumo.parquet"
)
parquet_source.timestamp_field = 'event_timestamp'

# Define a feature view
feature_view = FeatureView(
    name="evolucao_mensal",
    entities=[estado],
    features = lst_features,
    online=True,
    source=parquet_source
)

TypeError: FeatureView.__init__() got an unexpected keyword argument 'features'

In [7]:
from feast import FeatureStore

# Inicialize a feature store
fs = FeatureStore(repo_path="..\\MVPmodeloES\\my_repo\\feature_repo")

# Registrar as feature views
fs.apply([feature_view])

c:\Users\PC\miniconda3\lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [8]:
entity_df = df_hist_energia[['Estado','event_timestamp']]

In [13]:
dd

0      2017-01-01
1      2017-02-01
2      2017-03-01
3      2017-04-01
4      2017-05-01
          ...    
1696   2023-11-01
1697   2023-12-01
1698   2024-01-01
1699   2024-02-01
1700   2024-03-01
Name: event_timestamp, Length: 1701, dtype: datetime64[ns]

In [9]:
training_df = fs.get_historical_features(
    entity_df=entity_df,
    features=[
        "evolucao_mensal:EnergiaConsumida"
    ],
).to_df()

c:\Users\PC\miniconda3\lib\site-packages\feast\feature_view.py:417: UserWarning: There are some mismatches in your feature view's registered entities. Please check if you have applied your entities correctly.Entities: ['Estado'] vs Entity Columns: []
  warnings.warn(
c:\Users\PC\miniconda3\lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
c:\Users\PC\miniconda3\lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
c:\Users\PC\miniconda3\lib\site-packages\pyarrow\pandas_compat.py:373: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
c:\Users\PC\miniconda3\lib\site-packages\pyarrow\pandas_compat

In [10]:
training_df

,Estado,event_timestamp,EnergiaConsumida
0,Acre,2017-01-01 00:00:00+00:00,4.897130e+08
1,Acre,2017-02-01 00:00:00+00:00,1.624275e+09
2,Rio Grande do Norte,2017-03-01 00:00:00+00:00,7.858576e+09
3,Paraíba,2017-04-01 00:00:00+00:00,2.969244e+07
4,Rio de Janeiro,2017-05-01 00:00:00+00:00,3.023188e+08
...,...,...,...
58,Amapá,2023-11-01 00:00:00+00:00,9.411453e+08
59,Mato Grosso do Sul,2023-12-01 00:00:00+00:00,1.090770e+08
60,Rio de Janeiro,2024-01-01 00:00:00+00:00,2.128859e+09
61,Pará,2024-02-01 00:00:00+00:00,3.971758e+08


In [12]:
training_df[training_df['Estado']=='Rio de Janeiro']

,Estado,event_timestamp,EnergiaConsumida
4,Rio de Janeiro,2017-05-01 00:00:00+00:00,3.023188e+08
60,Rio de Janeiro,2024-01-01 00:00:00+00:00,2.128859e+09


In [11]:
entity_df

,Estado,event_timestamp
0,Acre,2017-01-01
1,Acre,2017-02-01
2,Acre,2017-03-01
3,Acre,2017-04-01
4,Acre,2017-05-01
...,...,...
1696,Tocantis,2023-11-01
1697,Tocantis,2023-12-01
1698,Tocantis,2024-01-01
1699,Tocantis,2024-02-01


In [45]:
training_df[training_df['Estado']=='Paraíba']

,Estado,event_timestamp,EnergiaConsumida
3,Paraíba,2017-04-01 00:00:00+00:00,29692440.0


In [46]:
training_df

,Estado,event_timestamp,EnergiaConsumida
0,Acre,2017-01-01 00:00:00+00:00,4.897130e+08
1,Acre,2017-02-01 00:00:00+00:00,1.624275e+09
2,Rio Grande do Norte,2017-03-01 00:00:00+00:00,7.858576e+09
3,Paraíba,2017-04-01 00:00:00+00:00,2.969244e+07
4,Rio de Janeiro,2017-05-01 00:00:00+00:00,3.023188e+08
...,...,...,...
58,Amapá,2023-11-01 00:00:00+00:00,9.411453e+08
59,Mato Grosso do Sul,2023-12-01 00:00:00+00:00,1.090770e+08
60,Rio de Janeiro,2024-01-01 00:00:00+00:00,2.128859e+09
61,Pará,2024-02-01 00:00:00+00:00,3.971758e+08


In [4]:
df_hist_energia.dropna(inplace = True)
data_corte = '2023-06-01'
df_train = df_hist_energia[df_hist_energia['MesReferencia']< data_corte]
df_test = df_hist_energia[df_hist_energia['MesReferencia']>= data_corte]
model = CatBoostRegressor()

features = ['EnergiaConsumida', 'EnergiaInjetada',
       'QtdConsumidores', 'Receita', 'TicketMédio', 'TakeRate',
       'ConsumoMédio']

model.fit( X= df_train[features], y = df_train['Target'])


y_pred = model.predict(df_test[features])
y_test = df_test['Target']
# Calcular métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# Exibir métricas
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')
print(f'MAPE: {mape}%')

Learning rate set to 0.042513
0:	learn: 51743535.7158417	total: 88.5ms	remaining: 1m 28s
1:	learn: 50046638.2150254	total: 89.7ms	remaining: 44.7s
2:	learn: 48387453.2047757	total: 90.7ms	remaining: 30.1s
3:	learn: 46814778.2610226	total: 91.6ms	remaining: 22.8s
4:	learn: 45336047.6800814	total: 92.8ms	remaining: 18.5s
5:	learn: 43961872.2756347	total: 94ms	remaining: 15.6s
6:	learn: 42552315.4730210	total: 95ms	remaining: 13.5s
7:	learn: 41167088.5867825	total: 96ms	remaining: 11.9s
8:	learn: 39943633.6889110	total: 97ms	remaining: 10.7s
9:	learn: 38703820.5419059	total: 97.8ms	remaining: 9.68s
10:	learn: 37513520.9030338	total: 98.8ms	remaining: 8.88s
11:	learn: 36294241.5481808	total: 99.9ms	remaining: 8.22s
12:	learn: 35201466.5464312	total: 101ms	remaining: 7.66s
13:	learn: 34075951.2982279	total: 102ms	remaining: 7.16s
14:	learn: 33055640.9823083	total: 103ms	remaining: 6.75s
15:	learn: 32076014.4990788	total: 104ms	remaining: 6.37s
16:	learn: 31074811.3081287	total: 105ms	remain

In [44]:
df_test['TargetPredicted'] = y_pred

C:\Users\PC\AppData\Local\Temp\ipykernel_6108\3719788760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['TargetPredicted'] = y_pred


In [50]:
#estados_sudeste = ['Rio de Janeiro','São Paulo','Minas Gerais']

#fig = px.area(df_test[df_test['Estado'].isin(estados_sudeste)], x = 'MesReferencia', y = 'Target', color = 'Estado', title = 'Predição')
#fig.show()

df_test[['Target','TargetPredicted']].head(2).to_dict(orient = 'records')

[{'Target': 5658455.0, 'TargetPredicted': 4920620.97908511},
 {'Target': 5348411.0, 'TargetPredicted': 5414514.262407333}]